# Scraping Twitter Using snscrape
<br>Package Github: https://github.com/JustAnotherArchivist/snscrape
<br>This notebook will be using the development version of snscrape

Article Read-Along: https://medium.com/better-programming/how-to-scrape-tweets-with-snscrape-90124ed006af

### Author: Martin Beck

<b>Dependencies: </b> 
- Your <b>Python</b> version must be <b>3.8</b> or higher. The development version of snscrape will not work with Python 3.7 or lower. You can download the latest Python version [here](https://www.python.org/downloads/).
- <b>Development version of snscrape</b>, uncomment the pip install line in the below cell to pip install in the notebook if you don't already have it.
- <b>Pandas</b>, the dataframes allows easy manipulation and indexing of data, this is more of a preference but is what I follow in this notebook.

In [1]:
#!pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Imports
import os
import pandas as pd
import itertools
import snscrape.modules.twitter as sntwitter

# Consulta por búsqueda de texto
El siguiente código buscará tweets a través de los siguientes términos:

* 'Dolor vacuna'
* 'efecto vacuna', 'efecto astrazeneca', etc.
* 'reacción vacuna'
* 'vacuna #AstraZeneca', 'vacuna #Sputnik', 'vacuna #j&j', etc.
* sputnik v, moderna
* 'vacuna sintoma'
* etc.

La búsqueda será de los años 2020, 2021 y 2022, y que serán almacenados en un archivo CSV

In [19]:
latitud = '19.309917325231165'   
longitud = '-99.12243737329997'
#radio de distancia a partir de 4
radio = '31.28km' 

localizacion = latitud + ',' + longitud + ',' + radio

#tweet_count = 100

#df_coord = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(
    #'efecto vacuna since:2022-01-01 until:2022-01-31 geocode:"{}"'.format(localizacion)).get_items(),tweet_count))[['id','url','date','content','likeCount','retweetCount']]

df_coord = pd.DataFrame((sntwitter.TwitterSearchScraper(
    'efecto sputnik since:2020-12-24 until:2022-06-17 geocode:"{}"'.format(localizacion)).get_items()))[['id','url','date','content','likeCount','retweetCount','replyCount']]

df_coord['latitud'] = latitud
df_coord['longitud'] = longitud
df_coord['radio'] = radio

df_coord.to_csv('efectos_covid.csv', sep=',', index=False,mode="w+")

# Pre procesamiento

In [1]:
import re
#!pip install emoji
import emoji
import pandas as pd
import numpy as np
#!pip install unidecode
from unidecode import unidecode

import nltk 
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
from nltk.corpus import stopwords

#!pip install pyspellchecker
from spellchecker import SpellChecker

from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#nltk.download('omw')
from nltk.corpus import wordnet

import spacy
#!python -m spacy download es
nlp = spacy.load('es_core_news_sm')

In [27]:
df_tw = pd.read_csv("post_vacuna.csv")
df_tw.head(100)

,id,url,date,content,likeCount,retweetCount,replyCount,latitud,longitud,radio
0,1.540000e+18,https://twitter.com/kikajurado/status/15416571...,2022-06-28 05:38:09+00:00,Quisiera saber si a alguien más le sigue dolie...,1,0,1,19.309917,-99.122437,31.28km
1,1.530000e+18,https://twitter.com/jey0029/status/15287510955...,2022-05-23 14:53:58+00:00,El Show de #ElPayasoLastimita\n\n¿Y la dónde q...,0,0,0,19.309917,-99.122437,31.28km
2,1.510000e+18,https://twitter.com/Nowhere_Girl/status/150827...,2022-03-28 03:00:00+00:00,Oigan alguien más también se sintió súper tris...,5,0,4,19.309917,-99.122437,31.28km
3,1.490000e+18,https://twitter.com/karixwitch/status/14939795...,2022-02-16 16:04:09+00:00,Alguien anotó las placas? Efecto post vacuna😷,0,0,1,19.309917,-99.122437,31.28km
4,1.490000e+18,https://twitter.com/alexlanip/status/149011584...,2022-02-06 00:11:17+00:00,@leonpalafox @Deloquenohay @taboada_mx @PPmeri...,2,0,1,19.309917,-99.122437,31.28km
5,1.490000e+18,https://twitter.com/verushhk/status/1489253732...,2022-02-03 15:05:33+00:00,Vengo de la oscuridad a decir que sobreviví a ...,5,0,2,19.309917,-99.122437,31.28km
6,1.490000e+18,https://twitter.com/NaokoLucero/status/1487640...,2022-01-30 04:14:59+00:00,"Por fin puedo tomar, ya paso el tiempo reglame...",2,0,1,19.309917,-99.122437,31.28km
7,1.490000e+18,https://twitter.com/aiturbejastrow/status/1486...,2022-01-26 00:30:21+00:00,¿Cuánto tiempo dura el estado de pendejez y le...,0,0,0,19.309917,-99.122437,31.28km
8,1.450000e+18,https://twitter.com/boliyoooo/status/144957192...,2021-10-17 03:04:13+00:00,"Encuesta post vacuna , opine !!! https://t.co/...",2,0,0,19.309917,-99.122437,31.28km
9,1.440000e+18,https://twitter.com/Alitzelcamacho/status/1443...,2021-09-30 21:45:03+00:00,Pinche sueño monumental que traigo post vacuna...,0,0,0,19.309917,-99.122437,31.28km


In [23]:
def remover_links(tweet):
    tweet = re.sub(r'http\S+', '', tweet)   # quitar http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # quitar bitly links
    tweet = tweet.strip('[link]')   # quitar [links]
    
    return tweet

def remover_menciones(tweet):
    tweet = re.sub(r"@\S+ ", "", tweet)
    
    return tweet

def rep(m):
    s = m.group(1)
    return ' '.join(re.split(r'(?=[A-Z])', s))

def separar_palabras_hashtag(tweet):
    tweet = re.sub(r'#(\w+)', rep, tweet)
    
    return tweet
    
def remover_hashtag(tweet):  
    tweet = re.sub(r'#', '', tweet)
    
    return tweet

def arreglar_abreviaciones(tweet):
    palabras = tweet.split()

    abreviaciones = {'d': 'de',
               'x': 'por',
               'xa': 'para',
               'as': 'has',
               'q': 'que',
               'k': 'que',
               'dl': 'del',
               'xq': 'porqué',
               'dr': 'doctor',
               'dra': 'doctora',
               'sr': 'señor',
               'sra': 'señora',
               'm': 'me',
               'aprox': 'aproximadamente',
               'hrs': 'horas',
                '1er': 'primer',    
                'jj': 'johnson'}
    
        
    tweet = " ".join([abreviaciones[palabra] if palabra in abreviaciones.keys() else palabra for palabra in palabras])

    return tweet

def arreglar_nombres_vacunas(tweet):
    i = 0
    
    palabras = tweet.split()
    
    for palabra in palabras:
        #normaliza el nombre de la vacuna inglesa
        if palabra == 'astra':
            palabras[i] = 'astrazeneca'
            
            if palabras[i + 1] == 'zeneca':
                del palabras[i + 1]
        
        if palabra == 'zeneca':
            palabras[i] = 'astrazeneca'
        
        i+=1
    
    return lista_a_cadena(palabras)
    
def remover_estilos(tweet):  
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    return tweet

def remover_numeros(tweet):
    tweet = re.sub(r'[0-9]', '', tweet)
    
    return tweet

def minusculas(tweet):
    tweet = tweet.lower()
    
    return tweet

def remover_puntuaciones(tweet):
    puntuaciones = '!"$%&\'()*+,-./:;<=>¿?[\\]^_`{|}~•@º'
    tweet = re.sub('[' + puntuaciones + ']+', ' ', tweet) 
    
    #Remover comillas
    tweet = tweet.replace("’", " ")
    tweet = tweet.replace("“", '')
    tweet = tweet.replace("”", '')
    
    #Remover puntos de suspensión
    tweet = tweet.replace('\u2026', '')
    
    return tweet

def remover_espacios(tweet):
    tweet = re.sub('\s+', ' ', tweet)
    
    return tweet

def remover_emojis(tweet):
    tweet = re.sub(emoji.get_emoji_regexp(), r"", tweet)
    
    return tweet

def remover_letras_repetidas(tweet):
    tweet = re.sub(r'(.)\1{2,}',r'\1', tweet)
    
    return tweet

def remover_signos_diacríticos(tweet):
    #Conversión de la letra ñ a n
    tweet = re.sub(u"[ñ]", 'n', tweet)
    
    #Remover signos diacríticos
    tweet = unidecode(tweet)
    
    return tweet

def tokenizar(tweet):
    tk = TweetTokenizer()
    tweet_tokenizado = tk.tokenize(tweet) 
    
    return tweet_tokenizado

def remover_stopwords(tweet_tokenizado):   
    stop_words = stopwords.words('spanish')
    tweet_tokenizado = [palabra for palabra in tweet_tokenizado if palabra not in stop_words]
    
    return tweet_tokenizado

def remover_único_token(tweet_tokenizado):
    a=list(tweet_tokenizado)
    
    for palabra in a:
        if len(palabra) == 1 :
            tweet_tokenizado.remove(palabra)
            
    return tweet_tokenizado

def corrector_otográfico(tweet):
    spell = SpellChecker(language='es')
    
    #Agrega nuevos términos al corrector ortográfico
    spell.word_frequency.load_text_file('terminos.txt')

    spell.word_frequency.load_words(['covid','sputnik', 'astrazeneca','pfizer','biontech','pfizerbiontech',
                                    'cansino','moderna','janssen','johnson','booster'])
    
    palabras = tweet.split()
    
    k = 0
    
    for palabra in palabras:
        palabra_correcta = spell.correction(palabra)
        palabras[k] = palabra_correcta
        
        k+=1 
        
    return palabras

def lista_a_cadena(s):
    str1 = " "
   
    return (str1.join(s))

def lematizar(tweet_tokenizado):
    lista = []
    doc = nlp(tweet_tokenizado)
    
    for token in doc:
        if str(token) == 'astrazeneca':
            lista.append('astrazeneca')
        else:
            lista.append(token.lemma_.lower())

    return lista       
                
    #lemas = [tok.lemma_.lower() for tok in doc]    
    #return lemas

In [32]:
content = df_tw['content']
lista_tokens = []

for i in range(len(content)):
    tweet = content[i]
    tweet = remover_links(tweet)
    tweet = remover_menciones(tweet)
    tweet = remover_estilos(tweet)
    tweet = separar_palabras_hashtag(tweet)
    tweet = minusculas(tweet)
    tweet = remover_numeros(tweet)
    tweet = remover_puntuaciones(tweet)
    tweet = remover_espacios(tweet)
    tweet = remover_emojis(tweet)
    tweet = remover_letras_repetidas(tweet)
    tweet = arreglar_abreviaciones(tweet)
    tweet = remover_signos_diacríticos(tweet)
    tweet = arreglar_nombres_vacunas(tweet) 
    tweet = corrector_otográfico(tweet)
    tweet = lista_a_cadena(tweet)
    tweet = remover_signos_diacríticos(tweet)
    tweet_tokenizado = tokenizar(tweet)
    tweet_tokenizado = remover_stopwords(tweet_tokenizado)
    tweet_tokenizado = lista_a_cadena(tweet_tokenizado)
    tweet_tokenizado = lematizar(tweet_tokenizado)
    tweet_tokenizado = remover_único_token(tweet_tokenizado) 
    
    lista_tokens.append(tweet_tokenizado)

C:\Users\erick\AppData\Local\Temp/ipykernel_6976/2925186884.py:108: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  tweet = re.sub(emoji.get_emoji_regexp(), r"", tweet)


In [35]:
df_tw['tweet_tokenizado'] = lista_tokens

df_tw.to_csv('post_vacuna.csv', sep=',', index=False,mode="w+")